-- plotlyをインストール

In [ ]:
alter session set use_cached_result = false;

In [ ]:
# setup for AI-powered enrichment
# Import required libraries (available in Snowflake notebooks)
import json
import re
import pandas as pd
from typing import List, Dict, Any
from snowflake.snowpark import Session

# Get the built-in Snowpark session
session = get_active_session()

# Configuration
HOURS_BACK = 12  # How many hours back to look in query history
SEMANTIC_VIEW_NAME = 'HR_SEMANTIC_VIEW'
CORTEX_MODEL = 'claude-4-sonnet'  # Claude model with high token limit

# Set context for the analysis
session.sql("USE ROLE agentic_analytics_vhol_role").collect()
session.sql("USE DATABASE SV_VHOL_DB").collect()
session.sql("USE SCHEMA VHOL_SCHEMA").collect()

# Verify connection
current_context = session.sql("""
    SELECT 
        CURRENT_DATABASE() as database,
        CURRENT_SCHEMA() as schema,
        CURRENT_WAREHOUSE() as warehouse,
        CURRENT_ROLE() as role,
        CURRENT_USER() as user
""").collect()

current_context

In [ ]:
# Query to retrieve VHOL Seed Queries from history
query_alter_timezone = f""" ALTER SESSION SET TIMEZONE = 'Asia/Tokyo' """

query_history_sql = f"""
SELECT 
    QUERY_TEXT,
    START_TIME,
    EXECUTION_STATUS,
    USER_NAME
FROM 
    SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
WHERE 
    START_TIME >= DATEADD('hour', -{HOURS_BACK}, CURRENT_TIMESTAMP())
    AND QUERY_TEXT ILIKE '%VHOL_Seed_Query%'
    AND QUERY_TEXT NOT ILIKE '%QUERY_TEXT%'
    AND EXECUTION_STATUS = 'SUCCESS'
ORDER BY 
    START_TIME DESC
LIMIT 50
"""

print(f"🔍 Retrieving VHOL Seed Queries from last {HOURS_BACK} hours...")

# Execute query and convert to pandas DataFrame
query_history_result = session.sql(query_alter_timezone)
query_history_result = session.sql(query_history_sql).collect()
query_history_df = pd.DataFrame([dict(row.asDict()) for row in query_history_result])

print(f"📊 Found {len(query_history_df)} VHOL Seed Queries in the last {HOURS_BACK} hours")

if len(query_history_df) > 0:
    print("\nSample queries found:")
    for i, row in query_history_df.head(3).iterrows():
        print(f"\n{i+1}. Query at {row['START_TIME']}:")
        # Show first 1000 characters of query
        query_preview = row['QUERY_TEXT'][:1000] + "..." if len(row['QUERY_TEXT']) > 1000 else row['QUERY_TEXT']
        print(f"   {query_preview}")
else:
    print("⚠️  No VHOL Seed Queries found. You may need to:")
    print("   1. Run some queries with 'VHOL Seed Query' comments")
    print("   2. Increase the HOURS_BACK parameter")
    print("   3. Check that the queries executed successfully")


In [ ]:
ALTER SESSION SET TIMEZONE = 'Asia/Tokyo';

SELECT 
    QUERY_TEXT,
    START_TIME,
    EXECUTION_STATUS,
    USER_NAME
FROM 
    SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
ORDER BY START_TIME desc
LIMIT 100
;

In [ ]:
ALTER SESSION SET TIMEZONE = 'Asia/Tokyo';

SELECT 
    QUERY_TEXT,
    START_TIME,
    EXECUTION_STATUS,
    USER_NAME
FROM 
    SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
WHERE 
    START_TIME >= DATEADD('hour', -12, CURRENT_TIMESTAMP())
    AND QUERY_TEXT ILIKE '%vhol_seed_query%'
    AND QUERY_TEXT NOT ILIKE '%QUERY_TEXT%'
    AND EXECUTION_STATUS = 'SUCCESS'
ORDER BY 
    START_TIME DESC
LIMIT 50

In [ ]:
def extract_metrics_and_dimensions(query_text: str) -> Dict[str, List[str]]:
    """
    Extract metrics (aggregation functions) and dimensions from SQL query
    """
    metrics = []
    dimensions = []
    
    # Clean query text
    query_clean = re.sub(r'--.*?\n', '\n', query_text)  # Remove line comments
    query_clean = re.sub(r'/\*.*?\*/', '', query_clean, flags=re.DOTALL)  # Remove block comments
    query_upper = query_clean.upper()
    
    # Extract aggregation functions (metrics)
    metric_patterns = [
        r'COUNT\s*\([^)]+\)',
        r'SUM\s*\([^)]+\)',
        r'AVG\s*\([^)]+\)',
        r'MIN\s*\([^)]+\)',
        r'MAX\s*\([^)]+\)',
        r'STDDEV\s*\([^)]+\)',
        r'PERCENTILE_CONT\s*\([^)]+\)',
        r'ROUND\s*\([^)]+\)',
    ]
    
    for pattern in metric_patterns:
        matches = re.findall(pattern, query_upper)
        metrics.extend(matches)
    
    # Extract column references from SELECT, WHERE, GROUP BY
    column_patterns = [
        r'SELECT\s+.*?([A-Z_][A-Z0-9_]*\.[A-Z_][A-Z0-9_]*)',  # table.column in SELECT
        r'WHERE\s+.*?([A-Z_][A-Z0-9_]*\.[A-Z_][A-Z0-9_]*)',   # table.column in WHERE
        r'GROUP BY\s+.*?([A-Z_][A-Z0-9_]*\.[A-Z_][A-Z0-9_]*)', # table.column in GROUP BY
        r'EXTRACT\s*\(\s*[A-Z]+\s+FROM\s+([A-Z_][A-Z0-9_]*\.[A-Z_][A-Z0-9_]*)\)',  # EXTRACT functions
        r'DATEDIFF\s*\([^,]+,\s*([A-Z_][A-Z0-9_]*\.[A-Z_][A-Z0-9_]*)',  # DATEDIFF functions
    ]
    
    for pattern in column_patterns:
        matches = re.findall(pattern, query_upper)
        for match in matches:
            # Skip if it's part of an aggregation function
            if not any(agg in match for agg in ['COUNT', 'SUM', 'AVG', 'MIN', 'MAX']):
                dimensions.append(match)
    
    # Clean and deduplicate
    metrics = list(set([m.strip() for m in metrics if m.strip()]))
    dimensions = list(set([d.strip() for d in dimensions if d.strip()]))
    
    return {
        'metrics': metrics,
        'dimensions': dimensions
    }

# Analyze all queries
all_metrics = []
all_dimensions = []

print("🔍 Analyzing queries for metrics and dimensions...")

for i, row in query_history_df.iterrows():
    analysis = extract_metrics_and_dimensions(row['QUERY_TEXT'])
    all_metrics.extend(analysis['metrics'])
    all_dimensions.extend(analysis['dimensions'])

# Deduplicate and summarize
unique_metrics = list(set(all_metrics))
unique_dimensions = list(set(all_dimensions))

print(f"\n📈 Analysis Results (with aliases):")
print(f"   Total unique metrics found: {len(unique_metrics)}")
print(f"   Total unique dimensions found: {len(unique_dimensions)}")

if unique_metrics:
    print(f"\n🔢 Sample Metrics (with aliases):")
    for metric in unique_metrics[:5]:  # Show first 5
        print(f"   - {metric}")

if unique_dimensions:
    print(f"\n📊 Sample Dimensions (with aliases):")
    for dim in unique_dimensions[:5]:  # Show first 5
        print(f"   - {dim}")

# VHOL table alias mappings
alias_to_table = {
    'F': 'HR_EMPLOYEE_FACT',
    'E': 'EMPLOYEE_DIM', 
    'D': 'DEPARTMENT_DIM',
    'J': 'JOB_DIM',
    'L': 'LOCATION_DIM'
}

print(f"\n🔧 Resolving VHOL table aliases to actual table names...")
print(f"📋 Alias mappings: {alias_to_table}")

# Resolve aliases in metrics
resolved_metrics = []
for metric in unique_metrics:
    resolved_metric = metric
    for alias, table in alias_to_table.items():
        resolved_metric = resolved_metric.replace(f'{alias}.', f'{table}.')
    resolved_metrics.append(resolved_metric)

# Resolve aliases in dimensions
resolved_dimensions = []
for dim in unique_dimensions:
    if '.' in dim:
        table_alias = dim.split('.')[0]
        column_name = dim.split('.')[1]
        
        if table_alias in alias_to_table:
            resolved_dim = f"{alias_to_table[table_alias]}.{column_name}"
            resolved_dimensions.append(resolved_dim)
        else:
            resolved_dimensions.append(dim)
    else:
        resolved_dimensions.append(dim)

# Update with resolved names
unique_metrics = list(set(resolved_metrics))
unique_dimensions = list(set(resolved_dimensions))

print(f"\n✅ Final Analysis Results (aliases resolved):")
print(f"   📊 Resolved metrics: {len(unique_metrics)}")
print(f"   📏 Resolved dimensions: {len(unique_dimensions)}")

if unique_metrics:
    print(f"\n🔢 Final Resolved Metrics:")
    for metric in unique_metrics[:5]:
        print(f"   - {metric}")

if unique_dimensions:
    print(f"\n📊 Final Resolved Dimensions:")
    for dim in unique_dimensions[:5]:
        print(f"   - {dim}")

print(f"\n🎯 Ready for semantic view enhancement!")


In [ ]:
# Retrieve current semantic view DDL
print(f"📋 Retrieving DDL for {SEMANTIC_VIEW_NAME}...")

try:
    ddl_result = session.sql(f"SELECT GET_DDL('semantic_view','{SEMANTIC_VIEW_NAME}') as DDL").collect()
    
    if ddl_result and len(ddl_result) > 0:
        current_ddl = ddl_result[0]['DDL']
        print(f"✅ Retrieved DDL for {SEMANTIC_VIEW_NAME}")
        print(f"📝 DDL Length: {len(current_ddl)} characters")
        
        # Show first few lines
        ddl_lines = current_ddl.split('\n')
        print(f"\n📋 Preview (first 20 lines):")
        for i, line in enumerate(ddl_lines[:20]):
            print(f"   {i+1:2d}: {line}")
        
        if len(ddl_lines) > 20:
            print(f"   ... ({len(ddl_lines)-20} more lines)")
    else:
        print(f"❌ No DDL found for {SEMANTIC_VIEW_NAME}")
        current_ddl = ""
        
except Exception as e:
    print(f"❌ Error retrieving DDL: {e}")
    current_ddl = ""

if current_ddl:
    print(f"\n✅ DDL retrieval successful! Ready for AI enhancement.")
else:
    print(f"\n⚠️  No DDL available - you may need to create the semantic view first.")


In [ ]:
if current_ddl and (unique_metrics or unique_dimensions):
    # Create AI prompt for enhancement (optimized for token efficiency)
    top_metrics = unique_metrics[:10]  # Top 10 most important
    top_dimensions = unique_dimensions[:10]  # Top 10 most important
    
    prompt = f"""
次の CREATE SEMANTIC VIEW の DDL を、検出されたクエリパターンに基づいて
新しい METRICS / DIMENSIONS の定義を追加することで拡張してください。

【現在の DDL】
{current_ddl}

【追加したいメトリクス】
{', '.join(top_metrics)}

【追加したいディメンション】
{', '.join(top_dimensions)}

【ルール】
- 既存の内容は一切変更せず、そのまま残してください
- 重要: DDL 内のセクション順序は必ず FACTS(), DIMENSIONS(), METRICS() の順に保ってください
- すべての集計式（SUM, COUNT, AVG など）は METRICS() セクションのみに追加してください
- METRICS() セクションでの書式は
    table_name.metric_name AS AGG(expression) --- added with AI enhancement
  としてください
- FACTS() セクションはテーブル参照専用であり、集計式は含めないでください
- 集計を伴わない列参照は DIMENSIONS セクションに追加してください
- 出力には "WITH EXTENSION" セクションを含めないでください
- 追加した行には必ず
    --- added with AI enhancement
  というコメントを付けてください
- 解説文や説明文は一切出力せず、「完成した拡張後の DDL 全体のみ」を返してください
- 始まりの「``` sql」や終わりの「``` 」は不要です。

【正しい DDL 構造の例】
FACTS (table_references)
DIMENSIONS (column_references)
METRICS (
HR_EMPLOYEE_FACT.total_salary AS SUM(salary) --- added with AI enhancement
)

【出力】
拡張後の CREATE SEMANTIC VIEW DDL 全体をそのまま出力してください。
"""
    
    # Escape single quotes for SQL
    prompt_escaped = prompt.replace("'", "''")
    
    # Use CORTEX_COMPLETE to generate enhanced DDL
    cortex_sql = f"""
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        '{CORTEX_MODEL}',
        '{prompt_escaped}'
    ) as enhanced_ddl
    """
    
    print(f"🤖 Using CORTEX_COMPLETE with {CORTEX_MODEL} to enhance semantic view...")
    print("   This may take 30-60 seconds...")
    
    try:
        # Execute CORTEX_COMPLETE
        cortex_result = session.sql(cortex_sql).collect()
        
        if cortex_result and len(cortex_result) > 0:
            enhanced_ddl = cortex_result[0]['ENHANCED_DDL']
            print("\n✅ Successfully generated enhanced semantic view DDL!")
            
            # Show statistics
            original_lines = len(current_ddl.split('\n'))
            enhanced_lines = len(enhanced_ddl.split('\n'))
            
            print(f"📊 Enhancement Statistics:")
            print(f"   Original DDL: {original_lines} lines, {len(current_ddl)} characters")
            print(f"   Enhanced DDL: {enhanced_lines} lines, {len(enhanced_ddl)} characters")
            print(f"   Lines added: {enhanced_lines - original_lines}")
            
            # Count new metrics and dimensions by looking for AI enhancement comments
            ai_additions_count = enhanced_ddl.count('--- added with AI enhancement')
            
            print(f"   New metrics/dimensions added: {ai_additions_count}")
            
        else:
            print("❌ CORTEX_COMPLETE returned no result")
            enhanced_ddl = current_ddl
            
    except Exception as e:
        print(f"❌ Error with CORTEX_COMPLETE: {e}")
        enhanced_ddl = current_ddl
        
else:
    print("⚠️  Skipping enhancement - no DDL or no new metrics/dimensions found")
    enhanced_ddl = current_ddl if 'current_ddl' in locals() else ""

# Display enhanced DDL results
if 'enhanced_ddl' in locals() and enhanced_ddl:
    print("\n" + "="*80)
    print("COMPLETE ENHANCED SEMANTIC VIEW DDL")
    print("="*80)
    print("📝 COMPLETE DDL OUTPUT (no truncation):")
    print()
    print(enhanced_ddl)
    print()
    print("="*80)
    
    # Highlight the new AI-enhanced additions
    enhanced_lines = enhanced_ddl.split('\n')
    new_additions = [line for line in enhanced_lines if '--- added with AI enhancement' in line]
    
    if new_additions:
        print("\n🤖 AI-ENHANCED ADDITIONS:")
        print("-" * 50)
        for addition in new_additions:
            print(addition.strip())
    else:
        print("\n⚠️  No new additions detected in the enhanced DDL")
    
    print(f"\n💡 Next Steps:")
    print(f"   1. Review the enhanced DDL above")
    print(f"   2. Test the DDL in a development environment")
    print(f"   3. Deploy to production when ready")
    print(f"   4. Update documentation with new metrics/dimensions")
    
else:
    print("❌ No enhanced DDL available")

print("\n🎉 Analysis complete!")
if 'query_history_df' in locals():
    print(f"   • Analyzed {len(query_history_df)} queries from the last {HOURS_BACK} hours")
if 'unique_metrics' in locals():
    print(f"   • Found {len(unique_metrics)} unique metrics")
if 'unique_dimensions' in locals():
    print(f"   • Found {len(unique_dimensions)} unique dimensions")
print(f"   • Enhanced {SEMANTIC_VIEW_NAME} using {CORTEX_MODEL}")


In [ ]:
///バックアップSQL
create or replace semantic view HR_SEMANTIC_VIEW  
	tables (  
		SV_VHOL_DB.VHOL_SCHEMA.DEPARTMENT_DIM primary key (DEPARTMENT_KEY) comment='このテーブルには、組織の部門の記録が含まれています。各レコードは識別情報を持つ1つの部門を表します。',  
		SV_VHOL_DB.VHOL_SCHEMA.EMPLOYEE_DIM primary key (EMPLOYEE_KEY) comment='このテーブルには、基本的な人口統計学的情報と雇用情報を含む従業員の記録が含まれています。各レコードは1人の従業員を表し、個人情報と採用情報が含まれています。',  
		SV_VHOL_DB.VHOL_SCHEMA.HR_EMPLOYEE_FACT primary key (HR_FACT_ID) comment='このテーブルには、分析目的のためにファクトテーブルとして整理された従業員情報の記録が含まれています。各レコードは特定の時点での従業員を表し、部門、職務、場所の次元参照と給与および離職状況が含まれています。',  
		SV_VHOL_DB.VHOL_SCHEMA.JOB_DIM primary key (JOB_KEY) comment='このテーブルには、組織内の職位の記録が含まれています。各レコードは個別の職務を表し、職位の名称と階層レベルに関する情報が含まれています。',  
		SV_VHOL_DB.VHOL_SCHEMA.LOCATION_DIM primary key (LOCATION_KEY) comment='このテーブルには、次元モデリングの参照データとして使用される場所の記録が含まれています。各レコードは識別情報を持つ個別の場所を表します。'  
	)  
	relationships (  
		HR_EMPLOYEE_FACT_TO_DEPARTMENT_DIM as HR_EMPLOYEE_FACT(DEPARTMENT_KEY) references DEPARTMENT_DIM(DEPARTMENT_KEY),  
		HR_EMPLOYEE_FACT_TO_EMPLOYEE_DIM as HR_EMPLOYEE_FACT(EMPLOYEE_KEY) references EMPLOYEE_DIM(EMPLOYEE_KEY),  
		HR_EMPLOYEE_FACT_TO_JOB_DIM as HR_EMPLOYEE_FACT(JOB_KEY) references JOB_DIM(JOB_KEY),  
		HR_EMPLOYEE_FACT_TO_LOCATION_DIM as HR_EMPLOYEE_FACT(LOCATION_KEY) references LOCATION_DIM(LOCATION_KEY)  
	)  
	facts (  
		HR_EMPLOYEE_FACT.SALARY as SALARY comment='従業員の給与額。'  
	)  
	dimensions (  
		DEPARTMENT_DIM.DEPARTMENT_KEY as DEPARTMENT_KEY comment='次元テーブルの各部門の一意の数値識別子。',  
		DEPARTMENT_DIM.DEPARTMENT_NAME as DEPARTMENT_NAME comment='組織内の部門の名称。',  
		EMPLOYEE_DIM.EMPLOYEE_KEY as EMPLOYEE_KEY comment='次元テーブルの各従業員レコードの一意の数値識別子。',  
		EMPLOYEE_DIM.EMPLOYEE_NAME as EMPLOYEE_NAME comment='従業員のフルネーム。',  
		EMPLOYEE_DIM.GENDER as GENDER comment='従業員の性別区分。',  
		EMPLOYEE_DIM.HIRE_DATE as HIRE_DATE comment='従業員が組織に採用された日付。',  
		HR_EMPLOYEE_FACT.ATTRITION_FLAG as ATTRITION_FLAG comment='従業員が組織を離れたかどうかを示すフラグ。',  
		HR_EMPLOYEE_FACT.DEPARTMENT_KEY as DEPARTMENT_KEY comment='組織内の部門の一意の識別子。',  
		HR_EMPLOYEE_FACT.EMPLOYEE_KEY as EMPLOYEE_KEY comment='システム内の各従業員レコードの一意の識別子。',  
		HR_EMPLOYEE_FACT.HR_FACT_ID as HR_FACT_ID comment='各人事ファクトレコードの一意の識別子。',  
		HR_EMPLOYEE_FACT.JOB_KEY as JOB_KEY comment='組織内の職位の一意の識別子。',  
		HR_EMPLOYEE_FACT.LOCATION_KEY as LOCATION_KEY comment='従業員の勤務地の一意の識別子。',  
		HR_EMPLOYEE_FACT.DATE as DATE comment='DATE型のデータを保持する列。',  
		JOB_DIM.JOB_KEY as JOB_KEY comment='次元テーブルの各職務レコードの一意の識別子。',  
		JOB_DIM.JOB_TITLE as JOB_TITLE comment='従業員が保持する職位または役職。',  
		LOCATION_DIM.LOCATION_KEY as LOCATION_KEY comment='各場所レコードの一意の数値識別子。',  
		LOCATION_DIM.LOCATION_NAME as LOCATION_NAME comment='都市とそれに対応する州の名称。',  
		JOB_DIM.JOB_LEVEL as JOB_LEVEL comment='職務の階層レベル。' --- added with AI enhancement  
	)  
	metrics (  
		HR_EMPLOYEE_FACT.female_employee_count AS COUNT(DISTINCT CASE WHEN EMPLOYEE_DIM.GENDER = 'F' THEN HR_EMPLOYEE_FACT.EMPLOYEE_KEY END) comment='女性従業員数。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.total_employee_count AS COUNT(DISTINCT HR_EMPLOYEE_FACT.EMPLOYEE_KEY) comment='総従業員数。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.min_salary AS MIN(HR_EMPLOYEE_FACT.SALARY) comment='最低給与額。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.min_date AS MIN(HR_EMPLOYEE_FACT.DATE) comment='最小日付。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.max_date AS MAX(HR_EMPLOYEE_FACT.DATE) comment='最大日付。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.total_attrition AS SUM(HR_EMPLOYEE_FACT.ATTRITION_FLAG) comment='総離職者数。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.salary_stddev AS STDDEV(HR_EMPLOYEE_FACT.SALARY) comment='給与の標準偏差。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.total_records AS COUNT(*) comment='総レコード数。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.avg_tenure_days AS AVG(DATEDIFF('DAY', EMPLOYEE_DIM.HIRE_DATE, HR_EMPLOYEE_FACT.DATE)) comment='平均在職日数。', --- added with AI enhancement  
		HR_EMPLOYEE_FACT.distinct_department_count AS COUNT(DISTINCT HR_EMPLOYEE_FACT.DEPARTMENT_KEY) comment='異なる部門数。' --- added with AI enhancement  
	)  

In [ ]:
# Deploy the enhanced semantic view DDL
if 'enhanced_ddl' in locals() and enhanced_ddl and enhanced_ddl.strip():
    print("🚀 Deploying Enhanced Semantic View...")
    print("="*60)
    
    try:
        # First, drop the existing semantic view if it exists
        drop_sql = f"DROP SEMANTIC VIEW IF EXISTS {SEMANTIC_VIEW_NAME}"
        print(f"📋 Dropping existing semantic view: {SEMANTIC_VIEW_NAME}")
        session.sql(drop_sql).collect()
        print("   ✅ Existing semantic view dropped successfully")
        
        # Execute the enhanced DDL
        print(f"🔧 Creating enhanced semantic view...")
        session.sql(enhanced_ddl).collect()
        print("   ✅ Enhanced semantic view created successfully!")
        
        # Verify the deployment
        verification_sql = f"SHOW SEMANTIC VIEWS LIKE '{SEMANTIC_VIEW_NAME}'"
        result = session.sql(verification_sql).collect()
        
        if result:
            print(f"\n🎉 SUCCESS! Enhanced {SEMANTIC_VIEW_NAME} deployed successfully!")
            print(f"📊 Semantic view details:")
            for row in result:
                print(f"   Name: {row['name']}")
                print(f"   Database: {row['database_name']}")
                print(f"   Schema: {row['schema_name']}")
                print(f"   Created: {row['created_on']}")
        else:
            print(f"⚠️  Deployment completed but verification failed - please check manually")
            
        # Show what was added
        if '--- added with AI enhancement' in enhanced_ddl:
            additions_count = enhanced_ddl.count('--- added with AI enhancement')
            print(f"\n🤖 AI Enhancement Summary:")
            print(f"   • {additions_count} new metrics/dimensions added")
            print(f"   • All additions marked with '--- added with AI enhancement'")
            print(f"   • Ready for immediate use in analytics!")
        
    except Exception as e:
        print(f"❌ Error deploying semantic view: {e}")
        print(f"\n🔍 Troubleshooting:")
        print(f"   1. Check if you have CREATE SEMANTIC VIEW privileges")
        print(f"   2. Verify the DDL syntax above is correct")
        print(f"   3. Ensure all referenced tables exist")
        print(f"   4. Try running the DDL manually if needed")
        
else:
    print("⚠️  No enhanced DDL available for deployment")
    print("   Please run Step 5 first to generate the enhanced DDL")

print(f"\n" + "="*60)
print("🏁 SEMANTIC VIEW ENHANCEMENT WORKFLOW COMPLETE!")
print("="*60)


In [ ]:
# Interactive Semantic View Visualization - Streamlit App for Snowflake Notebooks
# Uses SHOW METRICS and SHOW DIMENSIONS to dynamically discover available metrics and dimensions
# 
# Usage in Snowflake Notebook:
# 1. Make sure you have created the HR_SEMANTIC_VIEW
# 2. Paste this code into a Streamlit cell
# 3. The app will automatically discover metrics and dimensions

import streamlit as st
import pandas as pd
import plotly.express as px

# Semantic view configuration - adjust if needed
SEMANTIC_VIEW_NAME = "HR_SEMANTIC_VIEW"
SEMANTIC_VIEW_SCHEMA = "SV_VHOL_DB.VHOL_SCHEMA"  # Full schema path
SEMANTIC_VIEW_FULL_NAME = f"{SEMANTIC_VIEW_SCHEMA}.{SEMANTIC_VIEW_NAME}"

def main():
    st.title("🎯 Semantic View Interactive Visualization")
    st.markdown(f"**Semantic View:** `{SEMANTIC_VIEW_FULL_NAME}`")
    
    # Check if session is available (Snowflake notebook context)
    if 'session' not in globals():
        st.error("❌ Snowflake session not available. Please run this in a Snowflake notebook.")
        st.info("💡 Make sure you're running this in a Snowflake notebook with `session` available")
        return
    
    # Extract available metrics and dimensions using SHOW commands
    @st.cache_data
    def get_options():
        """Get metrics and dimensions from semantic view using SHOW SEMANTIC METRICS/DIMENSIONS commands
        Returns: (metrics_list, dimensions_list, metrics_map, dimensions_map)
        where maps contain full_name -> short_name mappings
        """
        metrics = []
        dimensions = []
        metrics_map = {}  # full_name -> short_name
        dimensions_map = {}  # full_name -> short_name
        
        try:
            # Get metrics from semantic view
            show_metrics_sql = f"SHOW SEMANTIC METRICS IN {SEMANTIC_VIEW_FULL_NAME}"
            
            with st.spinner("🔍 Fetching metrics from semantic view..."):
                metrics_result = session.sql(show_metrics_sql).collect()
            
            if metrics_result and len(metrics_result) > 0:
                # Convert to DataFrame to inspect structure
                metrics_df = pd.DataFrame([dict(row.asDict()) for row in metrics_result])
                
                # Debug: Show available columns (first time only)
                if 'metrics_debug' not in st.session_state:
                    with st.expander("🔍 Metrics Result Structure (Debug)", expanded=False):
                        st.dataframe(metrics_df.head())
                        st.write(f"Columns: {list(metrics_df.columns)}")
                    st.session_state.metrics_debug = True
                
                # Extract metric names - try common column names
                metric_name_col = None
                table_name_col = None
                
                for col in ['name', 'metric_name', 'metric', 'METRIC_NAME', 'NAME']:
                    if col in metrics_df.columns:
                        metric_name_col = col
                        break
                
                # Try to find table name column
                for col in ['table_name', 'table', 'TABLE_NAME', 'TABLE', 'source_table', 'entity_name']:
                    if col in metrics_df.columns:
                        table_name_col = col
                        break
                
                if metric_name_col:
                    for _, row in metrics_df.iterrows():
                        metric_name = str(row[metric_name_col]).strip()
                        if pd.isna(metric_name) or not metric_name:
                            continue
                        
                        # Try to get table name
                        table_name = None
                        if table_name_col and table_name_col in row:
                            table_name = str(row[table_name_col]).strip()
                            if pd.isna(table_name) or not table_name:
                                table_name = None
                        
                        # Check if metric_name already contains table prefix (table.metric format)
                        if '.' in metric_name:
                            # Already has table prefix
                            full_name = metric_name
                            short_name = metric_name.split('.')[-1]
                            metrics.append(full_name)
                            metrics_map[full_name] = short_name
                        elif table_name:
                            # Create full name with table prefix
                            full_name = f"{table_name}.{metric_name}"
                            metrics.append(full_name)
                            metrics_map[full_name] = metric_name
                        else:
                            # If no table name, use just the metric name
                            metrics.append(metric_name)
                            metrics_map[metric_name] = metric_name
                else:
                    # Fallback: use first column
                    metrics_raw = metrics_df.iloc[:, 0].dropna().unique().tolist()
                    for metric in metrics_raw:
                        metrics.append(str(metric))
                        metrics_map[str(metric)] = str(metric)
            else:
                st.warning("⚠️ No metrics found in semantic view")
            
            # Get dimensions from semantic view
            show_dimensions_sql = f"SHOW SEMANTIC DIMENSIONS IN {SEMANTIC_VIEW_FULL_NAME}"
            
            with st.spinner("🔍 Fetching dimensions from semantic view..."):
                dimensions_result = session.sql(show_dimensions_sql).collect()
            
            if dimensions_result and len(dimensions_result) > 0:
                # Convert to DataFrame to inspect structure
                dimensions_df = pd.DataFrame([dict(row.asDict()) for row in dimensions_result])
                
                # Debug: Show available columns (first time only)
                if 'dimensions_debug' not in st.session_state:
                    with st.expander("🔍 Dimensions Result Structure (Debug)", expanded=False):
                        st.dataframe(dimensions_df.head())
                        st.write(f"Columns: {list(dimensions_df.columns)}")
                    st.session_state.dimensions_debug = True
                
                # Extract dimension names - try common column names
                dimension_name_col = None
                table_name_col = None
                
                for col in ['name', 'dimension_name', 'dimension', 'DIMENSION_NAME', 'NAME']:
                    if col in dimensions_df.columns:
                        dimension_name_col = col
                        break
                
                # Try to find table name column
                for col in ['table_name', 'table', 'TABLE_NAME', 'TABLE', 'source_table', 'entity_name']:
                    if col in dimensions_df.columns:
                        table_name_col = col
                        break
                
                if dimension_name_col:
                    for _, row in dimensions_df.iterrows():
                        dimension_name = str(row[dimension_name_col]).strip()
                        if pd.isna(dimension_name) or not dimension_name:
                            continue
                        
                        # Try to get table name
                        table_name = None
                        if table_name_col and table_name_col in row:
                            table_name = str(row[table_name_col]).strip()
                            if pd.isna(table_name) or not table_name:
                                table_name = None
                        
                        # Check if dimension_name already contains table prefix (table.dimension format)
                        if '.' in dimension_name:
                            # Already has table prefix
                            full_name = dimension_name
                            short_name = dimension_name.split('.')[-1]
                            dimensions.append(full_name)
                            dimensions_map[full_name] = short_name
                        elif table_name:
                            # Create full name with table prefix
                            full_name = f"{table_name}.{dimension_name}"
                            dimensions.append(full_name)
                            dimensions_map[full_name] = dimension_name
                        else:
                            # If no table name, use just the dimension name
                            dimensions.append(dimension_name)
                            dimensions_map[dimension_name] = dimension_name
                else:
                    # Fallback: use first column
                    dimensions_raw = dimensions_df.iloc[:, 0].dropna().unique().tolist()
                    for dim in dimensions_raw:
                        dimensions.append(str(dim))
                        dimensions_map[str(dim)] = str(dim)
            else:
                st.warning("⚠️ No dimensions found in semantic view")
            
            # Fallback values if nothing found
            if not metrics and not dimensions:
                st.error("❌ Could not retrieve metrics or dimensions. Using fallback values.")
                st.info("💡 Make sure the semantic view exists and is accessible")
                metrics = ["HR_EMPLOYEE_FACT.TOTAL_EMPLOYEES", "HR_EMPLOYEE_FACT.AVG_SALARY", 
                          "HR_EMPLOYEE_FACT.TOTAL_SALARY_COST", "HR_EMPLOYEE_FACT.ATTRITION_COUNT"]
                dimensions = ["DEPARTMENT_DIM.DEPARTMENT_NAME", "JOB_DIM.JOB_TITLE", 
                            "LOCATION_DIM.LOCATION_NAME", "EMPLOYEE_DIM.EMPLOYEE_NAME"]
                # Create mappings for fallback
                for m in metrics:
                    metrics_map[m] = m.split('.')[-1] if '.' in m else m
                for d in dimensions:
                    dimensions_map[d] = d.split('.')[-1] if '.' in d else d
            elif not metrics:
                st.warning("⚠️ No metrics found, using fallback")
                metrics = ["HR_EMPLOYEE_FACT.TOTAL_EMPLOYEES", "HR_EMPLOYEE_FACT.AVG_SALARY", 
                          "HR_EMPLOYEE_FACT.TOTAL_SALARY_COST"]
                for m in metrics:
                    metrics_map[m] = m.split('.')[-1] if '.' in m else m
            elif not dimensions:
                st.warning("⚠️ No dimensions found, using fallback")
                dimensions = ["DEPARTMENT_DIM.DEPARTMENT_NAME", "JOB_DIM.JOB_TITLE", 
                            "LOCATION_DIM.LOCATION_NAME"]
                for d in dimensions:
                    dimensions_map[d] = d.split('.')[-1] if '.' in d else d
            
        except Exception as e:
            st.error(f"❌ Error fetching metrics/dimensions: {str(e)}")
            st.info("💡 Using fallback values. Make sure the semantic view exists and is accessible.")
            # Fallback values
            metrics = ["HR_EMPLOYEE_FACT.TOTAL_EMPLOYEES", "HR_EMPLOYEE_FACT.AVG_SALARY", 
                      "HR_EMPLOYEE_FACT.TOTAL_SALARY_COST", "HR_EMPLOYEE_FACT.ATTRITION_COUNT"]
            dimensions = ["DEPARTMENT_DIM.DEPARTMENT_NAME", "JOB_DIM.JOB_TITLE", 
                        "LOCATION_DIM.LOCATION_NAME", "EMPLOYEE_DIM.EMPLOYEE_NAME"]
            # Create mappings for fallback
            for m in metrics:
                metrics_map[m] = m.split('.')[-1] if '.' in m else m
            for d in dimensions:
                dimensions_map[d] = d.split('.')[-1] if '.' in d else d
            import traceback
            with st.expander("🔍 Error Details"):
                st.code(traceback.format_exc(), language='python')
        
        # Remove duplicates while preserving order
        metrics = list(dict.fromkeys(metrics))
        dimensions = list(dict.fromkeys(dimensions))
        
        return metrics, dimensions, metrics_map, dimensions_map

    try:
        metrics, dimensions, metrics_map, dimensions_map = get_options()
        
        if not metrics or not dimensions:
            st.error("❌ Could not load metrics or dimensions. Please check the semantic view.")
            return
        
        # Create two columns for the dropdowns
        col1, col2 = st.columns(2)
        
        with col1:
            selected_metric_full = st.selectbox(
                "📊 Select Metric:",
                metrics,
                help="Choose a metric to visualize",
                index=0 if metrics else None
            )
        
        with col2:
            selected_dimension_full = st.selectbox(
                "📏 Select Dimension:",
                dimensions,
                help="Choose a dimension to group by",
                index=0 if dimensions else None
            )
        
        if selected_metric_full and selected_dimension_full:
            # Get short names for ORDER BY (without table prefix)
            selected_metric_short = metrics_map.get(selected_metric_full, selected_metric_full.split('.')[-1] if '.' in selected_metric_full else selected_metric_full)
            selected_dimension_short = dimensions_map.get(selected_dimension_full, selected_dimension_full.split('.')[-1] if '.' in selected_dimension_full else selected_dimension_full)
            
            # Configuration section
            st.markdown("---")
            st.subheader("⚙️ Visualization Configuration")
            
            col_config1, col_config2, col_config3, col_config4 = st.columns(4)
            
            with col_config1:
                limit_rows = st.number_input(
                    "📊 Number of Rows:",
                    min_value=1,
                    max_value=1000,
                    value=10,
                    step=1,
                    help="Limit the number of rows returned"
                )
            
            with col_config2:
                viz_type = st.selectbox(
                    "📈 Visualization Type:",
                    ["Table", "Vertical Bar", "Horizontal Bar", "Line", "Pie"],
                    index=1,  # Default to Vertical Bar
                    help="Choose the chart type"
                )
            
            with col_config3:
                sort_by = st.selectbox(
                    "🔀 Sort By:",
                    ["Metric", "Dimension"],
                    index=0,  # Default to Metric
                    help="Choose which column to sort by"
                )
            
            with col_config4:
                sort_direction = st.selectbox(
                    "⬆️ Sort Direction:",
                    ["DESC", "ASC"],
                    index=0,  # Default to DESC
                    help="Choose sort direction"
                )
            
            # Determine sort column
            if sort_by == "Metric":
                sort_column = selected_metric_short
            else:
                sort_column = selected_dimension_short
            
            # Generate semantic SQL using SEMANTIC_VIEW() function
            # Use full names (with table prefix) inside SEMANTIC_VIEW()
            # Use short names (without prefix) in ORDER BY outside SEMANTIC_VIEW()
            query_sql = f"""SELECT * FROM SEMANTIC_VIEW(
    {SEMANTIC_VIEW_FULL_NAME}
    DIMENSIONS {selected_dimension_full}
    METRICS {selected_metric_full}
) ORDER BY {sort_column} {sort_direction} LIMIT {limit_rows}"""
            
            # Show the generated SQL in an expander
            with st.expander("📋 View Generated Semantic SQL"):
                st.code(query_sql, language='sql')
            
            # Execute the query and create visualization
            try:
                with st.spinner("🔄 Executing query and creating visualization..."):
                    try:
                        result = session.sql(query_sql).collect()
                    except Exception as sql_error:
                        # If full name doesn't work, try with just the view name
                        if "SEMANTIC_VIEW" in str(sql_error).upper() or "syntax" in str(sql_error).lower():
                            st.info("💡 Trying with view name only (without schema qualification)...")
                            fallback_query = f"""SELECT * FROM SEMANTIC_VIEW(
    {SEMANTIC_VIEW_NAME}
    DIMENSIONS {selected_dimension_full}
    METRICS {selected_metric_full}
) ORDER BY {sort_column} {sort_direction} LIMIT {limit_rows}"""
                            result = session.sql(fallback_query).collect()
                            query_sql = fallback_query  # Update the query shown
                        else:
                            raise sql_error
                
                if result and len(result) > 0:
                    # Convert to DataFrame
                    df = pd.DataFrame([dict(row.asDict()) for row in result])
                    
                    # Clean column names
                    df.columns = [col.strip() for col in df.columns]
                    
                    # Ensure we have numeric data for the metric
                    if len(df.columns) >= 2:
                        # Try to convert metric column to numeric
                        metric_col = df.columns[1]
                        df[metric_col] = pd.to_numeric(df[metric_col], errors='coerce')
                    
                    # Determine which columns to use
                    x_col = df.columns[0]
                    y_col = df.columns[1] if len(df.columns) > 1 else selected_metric_short
                    
                    # Explicitly sort the dataframe to maintain SQL sort order
                    # This ensures Plotly respects the sort order
                    sort_col_in_df = None
                    if sort_by == "Metric":
                        sort_col_in_df = y_col
                    else:
                        sort_col_in_df = x_col
                    
                    # Sort dataframe to match SQL ORDER BY
                    ascending = (sort_direction == "ASC")
                    df = df.sort_values(by=sort_col_in_df, ascending=ascending).reset_index(drop=True)
                    
                    metric_name = selected_metric_short.replace('_', ' ').title()
                    dimension_name = selected_dimension_short.replace('_', ' ').title()
                    
                    # Create visualization based on selected type
                    if viz_type == "Table":
                        # Show table directly
                        st.dataframe(df, use_container_width=True)
                    else:
                        # Create chart based on type
                        if viz_type == "Vertical Bar":
                            # Create category order to preserve dataframe sort order
                            category_order = df[x_col].tolist()
                            fig = px.bar(
                                df, 
                                x=x_col, 
                                y=y_col,
                                title=f'{metric_name} by {dimension_name}',
                                labels={
                                    x_col: dimension_name,
                                    y_col: metric_name
                                },
                                color=y_col,
                                color_continuous_scale='Blues',
                                category_orders={x_col: category_order}
                            )
                            fig.update_layout(
                                showlegend=False,
                                height=500,
                                xaxis_tickangle=-45,
                                hovermode='x unified',
                                xaxis={'categoryorder': 'array', 'categoryarray': category_order}
                            )
                        
                        elif viz_type == "Horizontal Bar":
                            # For horizontal bars, preserve y-axis (category) order
                            category_order = df[x_col].tolist()
                            fig = px.bar(
                                df, 
                                x=y_col,
                                y=x_col,
                                orientation='h',
                                title=f'{metric_name} by {dimension_name}',
                                labels={
                                    x_col: dimension_name,
                                    y_col: metric_name
                                },
                                color=y_col,
                                color_continuous_scale='Blues',
                                category_orders={x_col: category_order}
                            )
                            fig.update_layout(
                                showlegend=False,
                                height=max(400, len(df) * 30),  # Dynamic height based on rows
                                hovermode='y unified',
                                yaxis={'categoryorder': 'array', 'categoryarray': category_order}
                            )
                        
                        elif viz_type == "Line":
                            # Preserve x-axis order for line charts
                            category_order = df[x_col].tolist()
                            fig = px.line(
                                df, 
                                x=x_col, 
                                y=y_col,
                                title=f'{metric_name} by {dimension_name}',
                                labels={
                                    x_col: dimension_name,
                                    y_col: metric_name
                                },
                                markers=True,
                                category_orders={x_col: category_order}
                            )
                            fig.update_layout(
                                height=500,
                                xaxis_tickangle=-45,
                                hovermode='x unified',
                                xaxis={'categoryorder': 'array', 'categoryarray': category_order}
                            )
                        
                        elif viz_type == "Pie":
                            fig = px.pie(
                                df,
                                values=y_col,
                                names=x_col,
                                title=f'{metric_name} by {dimension_name}'
                            )
                            fig.update_layout(
                                height=500,
                                showlegend=True
                            )
                            fig.update_traces(textposition='inside', textinfo='percent+label')
                        
                        st.plotly_chart(fig, use_container_width=True)
                    
                    # Show data table in expander (always available)
                    with st.expander("📊 View Data Table"):
                        st.dataframe(df, use_container_width=True)
                    
                    # Show query execution info
                    with st.expander("🔍 Query Execution Details"):
                        st.code(query_sql, language='sql')
                        st.write(f"**Rows returned:** {len(df)}")
                        st.write(f"**Columns:** {', '.join(df.columns)}")
                        if len(df.columns) >= 2:
                            st.write(f"**Metric range:** {df[y_col].min():,.2f} to {df[y_col].max():,.2f}")
                    
                    st.success(f"✅ Successfully visualized {len(df)} data points!")
                    
                else:
                    st.warning("⚠️ No data returned from the semantic view query")
                    st.info("💡 Try selecting different metrics or dimensions")
                    
            except Exception as e:
                st.error(f"❌ Error executing query: {str(e)}")
                st.info("💡 Troubleshooting tips:")
                st.info("1. Make sure the semantic view exists and is accessible")
                st.info("2. Verify you have proper permissions to query the semantic view")
                st.info("3. Check that the metric and dimension names are correct")
                st.info("4. Try the SQL query manually in a SQL cell to debug")
                import traceback
                with st.expander("🔍 Error Details"):
                    st.code(traceback.format_exc(), language='python')
    
    except Exception as e:
        st.error(f"❌ Error loading options: {str(e)}")
        st.info("💡 Make sure the semantic view was created successfully")
        import traceback
        with st.expander("🔍 Error Details"):
            st.code(traceback.format_exc(), language='python')

# Run the Streamlit app
if __name__ == "__main__":
    main()




In [ ]:
# Natural Language Query Interface for Semantic Views
# Streamlit App for Snowflake Notebooks
# Uses Cortex Analyst REST API
# 
# Usage in Snowflake Notebook:
# 1. Make sure you're in a Snowflake notebook (not local Streamlit)
# 2. The 'session' variable should be automatically available
# 3. Paste this code into a Streamlit cell
# 4. Select a semantic view from the dropdown
# 5. Type your natural language question
# 6. Click "Answer!" to execute
#
# Note: If session is not available, ensure you're running in a Snowflake notebook environment.
# The session variable is created automatically when you run a SQL cell in a Snowflake notebook.

import streamlit as st
import pandas as pd
import json
import time

# Try to import _snowflake (available in Snowflake notebooks)
try:
    import _snowflake  # For interacting with Snowflake-specific APIs
    SNOWFLAKE_API_AVAILABLE = True
except ImportError:
    SNOWFLAKE_API_AVAILABLE = False
    _snowflake = None

# Schema configuration - adjust if needed
DEFAULT_SCHEMA = "SV_VHOL_DB.VHOL_SCHEMA"

def make_authenticated_request_via_session(session, url, method="POST", json_data=None, headers=None):
    """
    Attempt to make an HTTP request using the session's connection
    This bypasses the need for explicit OAuth token extraction
    """
    try:
        # Try to get the connection object
        conn = None
        if hasattr(session, '_conn'):
            conn = session._conn
        elif hasattr(session, 'connection'):
            conn = session.connection
        
        if not conn:
            return None
        
        # Try different methods to make HTTP requests through the connection
        # Method 1: Check if connection has an HTTP client or request method
        if hasattr(conn, '_request') or hasattr(conn, 'request'):
            request_method = getattr(conn, '_request', None) or getattr(conn, 'request', None)
            if request_method:
                try:
                    # Try to make the request
                    response = request_method(url, method=method, json=json_data, headers=headers)
                    return response
                except:
                    pass
        
        # Method 2: Check if there's an HTTP client or session object
        if hasattr(conn, '_http') or hasattr(conn, 'http') or hasattr(conn, '_session') or hasattr(conn, 'session'):
            http_client = (getattr(conn, '_http', None) or 
                          getattr(conn, 'http', None) or
                          getattr(conn, '_session', None) or
                          getattr(conn, 'session', None))
            if http_client:
                try:
                    if method == "POST":
                        response = http_client.post(url, json=json_data, headers=headers)
                    else:
                        response = http_client.request(method, url, json=json_data, headers=headers)
                    return response
                except:
                    pass
        
    except Exception:
        pass
    
    return None

def generate_oauth_token_from_session(session, account, region):
    """
    Attempt to generate an OAuth token using the current session
    This uses Snowflake's OAuth API to create a token for REST API calls
    """
    try:
        # Try to use Snowflake's OAuth token generation
        # Note: SYSTEM$GENERATE_OAUTH_TOKEN might not be available
        try:
            token_result = session.sql("SELECT SYSTEM$GENERATE_OAUTH_TOKEN() as token").collect()
            if token_result and len(token_result) > 0:
                token = token_result[0].get('TOKEN')
                if token:
                    return token
        except:
            # SYSTEM$GENERATE_OAUTH_TOKEN might not be available
            pass
        
    except Exception as e:
        # Silently fail
        pass
    
    return None

def get_auth_token(session):
    """Try to extract authentication token from Snowflake session"""
    auth_token = None
    
    def _check_object_for_token(obj, depth=0, max_depth=3):
        """Recursively search an object for token-like values"""
        if depth > max_depth or obj is None:
            return None
        
        # Check direct token attributes
        token_attrs = ['_token', 'token', '_master_token', 'master_token', '_session_token', 
                      'session_token', 'access_token', '_access_token', 'bearer_token', '_bearer_token']
        for attr in token_attrs:
            if hasattr(obj, attr):
                try:
                    value = getattr(obj, attr)
                    if value and isinstance(value, str) and len(value) > 20:  # Tokens are usually long strings
                        return value
                except:
                    pass
        
        # Check if it's a dict-like object
        if hasattr(obj, '__dict__'):
            for key, value in obj.__dict__.items():
                if 'token' in key.lower() and isinstance(value, str) and len(value) > 20:
                    return value
                # Recursively check nested objects (but limit depth)
                if depth < max_depth and isinstance(value, object) and not isinstance(value, (str, int, float, bool)):
                    result = _check_object_for_token(value, depth + 1, max_depth)
                    if result:
                        return result
        
        return None
    
    try:
        # Try to get from session's connection
        conn = None
        
        # Method 1: Try session._conn (Snowpark)
        if hasattr(session, '_conn'):
            conn = session._conn
        # Method 2: Try session.connection (alternative attribute name)
        elif hasattr(session, 'connection'):
            conn = session.connection
        # Method 3: Try session._connection (another variant)
        elif hasattr(session, '_connection'):
            conn = session._connection
        
        if conn:
            # Method A: Try REST client token (for Python connector connections)
            if hasattr(conn, '_rest'):
                rest_client = conn._rest
                # Try direct attributes first
                for token_attr in ['_token', 'token', '_master_token', 'master_token', '_session_token']:
                    if hasattr(rest_client, token_attr):
                        try:
                            token_value = getattr(rest_client, token_attr)
                            if token_value and isinstance(token_value, str) and len(token_value) > 20:
                                auth_token = token_value
                                break
                        except:
                            pass
                
                # Try recursive search if direct access failed
                if not auth_token:
                    auth_token = _check_object_for_token(rest_client, max_depth=2)
                
                # Try token manager if available
                if not auth_token and hasattr(rest_client, '_token_manager'):
                    token_manager = rest_client._token_manager
                    auth_token = _check_object_for_token(token_manager, max_depth=2)
            
            # Method A2: For ServerConnection (Snowflake notebooks), try different attributes
            # ServerConnection might have token stored differently
            if not auth_token:
                # Try connection-level token attributes
                auth_token = _check_object_for_token(conn, max_depth=3)
            
            # Method A3: Try to get from connection's internal state
            if not auth_token:
                # Check for session token or authentication state
                internal_attrs = ['_session_token', '_auth_token', '_token', 'token', 
                                 '_session', '_authenticator', '_login_manager']
                for attr in internal_attrs:
                    if hasattr(conn, attr):
                        try:
                            value = getattr(conn, attr)
                            if isinstance(value, str) and len(value) > 20:
                                auth_token = value
                                break
                            elif hasattr(value, '__dict__'):
                                # If it's an object, search it recursively
                                token = _check_object_for_token(value, max_depth=2)
                                if token:
                                    auth_token = token
                                    break
                        except:
                            pass
            
            # Method B: Try connection-level token attributes (recursive)
            if not auth_token:
                auth_token = _check_object_for_token(conn, max_depth=3)
            
            # Method C: Try from connection's authentication handler
            if not auth_token:
                auth_attrs = ['_authenticate', '_auth', 'authenticate', '_auth_handler', 'auth_handler']
                for auth_attr in auth_attrs:
                    if hasattr(conn, auth_attr):
                        try:
                            auth_handler = getattr(conn, auth_attr)
                            auth_token = _check_object_for_token(auth_handler, max_depth=2)
                            if auth_token:
                                break
                        except:
                            pass
            
            # Method D: Try to get from connection's headers/cookies
            if not auth_token and hasattr(conn, '_rest'):
                rest_client = conn._rest
                # Check if there's a headers dict with authorization
                header_attrs = ['_headers', 'headers', '_request_headers', 'request_headers']
                for header_attr in header_attrs:
                    if hasattr(rest_client, header_attr):
                        try:
                            headers = getattr(rest_client, header_attr)
                            if isinstance(headers, dict):
                                auth_header = headers.get('Authorization') or headers.get('authorization')
                                if auth_header and isinstance(auth_header, str):
                                    if auth_header.startswith('Bearer '):
                                        auth_token = auth_header[7:]  # Remove 'Bearer ' prefix
                                    else:
                                        auth_token = auth_header
                                    if auth_token:
                                        break
                        except:
                            pass
    
    except Exception as e:
        # Silently fail - we'll handle missing token in the UI
        pass
    
    return auth_token

def main():
    st.title("💬 Natural Language Query for Semantic Views")
    st.markdown("Ask questions in plain English about your semantic view data")
    st.markdown("*Using [Cortex Analyst REST API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/rest-api)*")
    
    # Check if session is available (Snowflake notebook context)
    # In Snowflake notebooks, session is typically available as a global variable
    if 'session' not in globals():
        st.error("❌ Snowflake session not available. Please run this in a Snowflake notebook.")
        st.info("💡 Make sure you're running this in a Snowflake notebook with `session` available")
        return
    
    # Get account and region info early - cache it for the session
    @st.cache_data
    def get_account_info():
        """Get account and region from the current Snowflake session"""
        try:
            account_info = session.sql("SELECT CURRENT_ACCOUNT() as account, CURRENT_REGION() as region").collect()
            if account_info and len(account_info) > 0:
                account = account_info[0]['ACCOUNT']
                region = account_info[0]['REGION']
                return account, region
        except Exception:
            pass
        return None, None
    
    # Pre-populate account and region first (needed for token generation)
    account, region = get_account_info()
    
    # Get token early - cache it for the session
    @st.cache_data
    def get_cached_token(account_val, region_val):
        """Get auth token from session - cached, tries extraction then generation"""
        # First try to extract existing token
        token = get_auth_token(session)
        
        # If extraction failed and we have account/region, try generating one
        if not token and account_val and region_val:
            try:
                token = generate_oauth_token_from_session(session, account_val, region_val)
            except:
                pass
        
        return token
    
    # Check if _snowflake API is available (required for authentication)
    if account and region:
        if not SNOWFLAKE_API_AVAILABLE:
            st.error("⚠️ `_snowflake` module not available. This app requires running in a Snowflake notebook.")
            st.info("💡 The `_snowflake` module provides automatic authentication for REST API calls.")
            return
    else:
        st.warning("⚠️ Could not retrieve account information. Some features may not work.")
    
    # Get available semantic views in the schema
    @st.cache_data
    def get_semantic_views(schema_name):
        """Get list of available semantic views in the schema"""
        try:
            # Handle schema name (could be "DATABASE.SCHEMA" or just "SCHEMA")
            if '.' in schema_name:
                database, schema = schema_name.split('.', 1)
                show_sql = f"SHOW SEMANTIC VIEWS IN SCHEMA {database}.{schema}"
            else:
                # Try to use current database context
                show_sql = f"SHOW SEMANTIC VIEWS IN SCHEMA {schema_name}"
            
            result = session.sql(show_sql).collect()
            
            if result and len(result) > 0:
                # Convert to DataFrame
                views_df = pd.DataFrame([dict(row.asDict()) for row in result])
                
                # Try to find the name column
                name_col = None
                for col in ['name', 'semantic_view_name', 'view_name', 'NAME', 'SEMANTIC_VIEW_NAME']:
                    if col in views_df.columns:
                        name_col = col
                        break
                
                if name_col:
                    views = views_df[name_col].dropna().unique().tolist()
                else:
                    # Fallback: use first column
                    views = views_df.iloc[:, 0].dropna().unique().tolist()
                
                # Create full qualified names
                full_names = []
                for view in views:
                    full_name = f"{schema_name}.{view}" if '.' not in view else view
                    full_names.append(full_name)
                
                return full_names, views_df
            else:
                return [], pd.DataFrame()
                
        except Exception as e:
            st.error(f"❌ Error fetching semantic views: {str(e)}")
            return [], pd.DataFrame()
    
    # Schema selection
    schema_input = st.text_input(
        "📁 Schema:",
        value=DEFAULT_SCHEMA,
        help="Enter the schema path (e.g., DATABASE.SCHEMA)"
    )
    
    # Get semantic views
    with st.spinner("🔍 Loading semantic views..."):
        semantic_views, views_df = get_semantic_views(schema_input)
    
    if not semantic_views:
        st.warning(f"⚠️ No semantic views found in {schema_input}")
        st.info("💡 Make sure the schema name is correct and contains semantic views")
        
        # Show debug info if available
        if not views_df.empty:
            with st.expander("🔍 Debug: SHOW SEMANTIC VIEWS Result"):
                st.dataframe(views_df)
        return
    
    # Semantic view selection
    selected_view = st.selectbox(
        "📊 Select Semantic View:",
        semantic_views,
        help="Choose a semantic view to query",
        index=0 if semantic_views else None
    )
    
    if selected_view:
        st.markdown("---")
        
        # Natural language question input
        st.subheader("💬 Ask Your Question")
        question = st.text_area(
            "Enter your question:",
            height=100,
            placeholder="e.g., What are the top 5 departments by average salary?",
            help="Type your question in natural language"
        )
        
        # Answer button
        col1, col2 = st.columns([1, 4])
        with col1:
            answer_button = st.button("🚀 Answer!", type="primary", use_container_width=True)
        
        if answer_button and question:
            if not question.strip():
                st.warning("⚠️ Please enter a question")
            else:
                # Generate SQL from natural language question using Cortex Analyst REST API
                generated_sql = None  # Initialize outside try block
                
                try:
                    with st.spinner("🤖 Generating SQL from your question..."):
                        # Use Snowflake's built-in API request method (no token needed!)
                        if not SNOWFLAKE_API_AVAILABLE:
                            st.error("❌ `_snowflake` module not available. Make sure you're running this in a Snowflake notebook.")
                            st.info("💡 The `_snowflake` module is automatically available in Snowflake notebooks.")
                            return
                        
                        # Build request body for Cortex Analyst API
                        # According to Snowflake Labs example: https://github.com/Snowflake-Labs/sfguide-getting-started-with-cortex-analyst
                        # Note: API requires exactly one of: semantic_model, semantic_model_file, or semantic_view
                        request_body = {
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [
                                        {
                                            "type": "text",
                                            "text": question
                                        }
                                    ]
                                }
                            ],
                            "semantic_view": selected_view
                        }
                        
                        # Use Snowflake's built-in API request method
                        # This automatically handles authentication - no token needed!
                        API_ENDPOINT = "/api/v2/cortex/analyst/message"
                        API_TIMEOUT = 50000  # in milliseconds
                        
                        resp = _snowflake.send_snow_api_request(
                            "POST",  # method
                            API_ENDPOINT,  # path
                            {},  # headers (empty - auth is handled automatically)
                            {},  # params
                            request_body,  # body
                            None,  # request_guid
                            API_TIMEOUT,  # timeout in milliseconds
                        )
                        
                        # Parse response
                        # Content is a string with serialized JSON object
                        parsed_content = json.loads(resp["content"])
                        
                        # Check if the response is successful
                        if resp["status"] >= 400:
                            # Error response
                            error_msg = f"""
🚨 An Analyst API error has occurred 🚨

* response code: `{resp['status']}`
* request-id: `{parsed_content.get('request_id', 'N/A')}`
* error code: `{parsed_content.get('error_code', 'N/A')}`

Message:

{parsed_content.get('message', 'Unknown error')}

                            """
                            st.error(error_msg)
                            generated_sql = None
                        else:
                            # Success - extract response data
                            response_data = parsed_content
                            
                            # Extract SQL from response
                            # Response structure: message.content[] with type "sql" containing "statement"
                            text_response = None
                            
                            if 'message' in response_data and 'content' in response_data['message']:
                                for content_block in response_data['message']['content']:
                                    if content_block.get('type') == 'sql':
                                        generated_sql = content_block.get('statement', '')
                                    elif content_block.get('type') == 'text':
                                        text_response = content_block.get('text', '')
                            
                            # Show text interpretation if available
                            if text_response:
                                with st.expander("📝 Interpretation", expanded=False):
                                    st.write(text_response)
                            
                            # Show warnings if any
                            if 'warnings' in response_data and response_data['warnings']:
                                for warning in response_data['warnings']:
                                    st.warning(f"⚠️ {warning.get('message', 'Warning')}")
                            
                            if generated_sql:
                                # Show generated SQL
                                with st.expander("🔍 Generated SQL Query", expanded=False):
                                    st.code(generated_sql, language='sql')
                                
                                # Show response metadata if available
                                if 'response_metadata' in response_data:
                                    with st.expander("📊 Response Metadata", expanded=False):
                                        st.json(response_data['response_metadata'])
                            else:
                                # Check if suggestions were provided
                                suggestions_found = False
                                if 'message' in response_data and 'content' in response_data['message']:
                                    for content_block in response_data['message']['content']:
                                        if content_block.get('type') == 'suggestions':
                                            st.info("💡 Your question might be ambiguous. Here are some suggestions:")
                                            suggestions = content_block.get('suggestions', [])
                                            for i, suggestion in enumerate(suggestions, 1):
                                                st.write(f"{i}. {suggestion}")
                                            suggestions_found = True
                                
                                if not suggestions_found:
                                    st.error("❌ No SQL generated. Check the response for details.")
                                    with st.expander("🔍 Full Response"):
                                        st.json(response_data)
                                    generated_sql = None  # Ensure it's None if no SQL generated
                        
                        # Execute the query if SQL was generated
                        if generated_sql:
                            with st.spinner("🔄 Executing query..."):
                                try:
                                    result = session.sql(generated_sql).collect()
                                    
                                    if result and len(result) > 0:
                                        # Convert to DataFrame
                                        df = pd.DataFrame([dict(row.asDict()) for row in result])
                                        
                                        # Display results
                                        st.subheader("📊 Results")
                                        st.dataframe(df, use_container_width=True)
                                        
                                        # Show summary
                                        st.success(f"✅ Query executed successfully! Returned {len(df)} rows.")
                                        
                                        # Show query details
                                        with st.expander("📋 Query Details"):
                                            st.code(generated_sql, language='sql')
                                            st.write(f"**Rows returned:** {len(df)}")
                                            st.write(f"**Columns:** {', '.join(df.columns)}")
                                        
                                    else:
                                        st.info("ℹ️ Query executed but returned no results.")
                                        
                                except Exception as e:
                                    st.error(f"❌ Error executing query: {str(e)}")
                                    st.info("💡 The generated SQL might need adjustment. Check the generated SQL above.")
                                    import traceback
                                    with st.expander("🔍 Error Details"):
                                        st.code(traceback.format_exc(), language='python')
                        
                        else:
                            st.error("❌ Could not generate SQL from Cortex Analyst API")
                            st.info("💡 Check the API response above for details.")
                    
                except Exception as e:
                    st.error(f"❌ Error generating SQL: {str(e)}")
                    st.info("💡 Make sure you're running in a Snowflake notebook and that Cortex Analyst is available in your account.")
                    import traceback
                    with st.expander("🔍 Error Details"):
                        st.code(traceback.format_exc(), language='python')
    
    # Show available semantic views info
    with st.expander("ℹ️ About This App"):
        st.markdown("""
        **How to use:**
        1. Select a semantic view from the dropdown
        2. Type your question in natural language
        3. Click "Answer!" to generate and execute the query
        
        **Example questions:**
        - "What are the top 10 departments by total employees?"
        - "Show me average salary by job title"
        - "Which locations have the highest attrition rates?"
        - "List the top 5 employees by salary"
        
        **Note:** This app uses the [Cortex Analyst REST API](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-analyst/rest-api) 
        to generate SQL from natural language questions. The API automatically understands your semantic view 
        structure and generates appropriate queries.
        
        **Authentication:** The app attempts to automatically retrieve your authentication token from the session.
        If that fails, you can manually enter an OAuth token when prompted.
        """)

# Run the Streamlit app
if __name__ == "__main__":
    main()


